<a href="https://colab.research.google.com/github/aditi6/NLP-Project/blob/main/llama_convo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llamaapi -q

In [ ]:
from llamaapi import LlamaAPI
import json
# please change the api to your own key
llama = LlamaAPI('LL-zUoQjcGaE0WvMKj0Wck7ZdocLqQPMCccjgodbEwUAQnf1GJku1T8Tav6w9wYGjPN')

In [ ]:
from llamaapi import LlamaAPI
import pandas as pd
import json

# Initialize the Llama API with your API token
# llama = LlamaAPI('your_llama_api_token')

def get_response_from_llama(prompt):
    # Build the API request JSON with the user's prompt
    api_request_json = {
        "model": "llama-13b-chat",
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }

    # Execute the API request
    response = llama.run(api_request_json)

    # Parse and print the JSON response
    if response.ok:
        return json.dumps(response.json(), indent=2)
    else:
        return "Error in Llama API response"

def main():
    # Load the prompts from the CSV file
    df = pd.read_csv('./generated_prompts.csv')

    # Initialize a new column for the responses
    df['response'] = ''

    # Process each prompt and get responses from Llama
    for index, row in df.iterrows():
        prompt = f"scenario:{row['scenario']}, character1:{row['character1']}, goal1:{row['goal1']}, character2:{row['character2']}, goal2:{row['goal2']}, task:{row['task']}"
        response = get_response_from_llama(prompt)
        df.at[index, 'response'] = response

    # Save the updated DataFrame back to the CSV
    df.to_csv('./generated_response.csv', index=False)

if __name__ == "__main__":
    main()


In [ ]:

llama1 = LlamaAPI('LL-1JjExK2KMuL9qH30YOKdUvtk98gtZ0umxNmT8IOoWXweIqGiWmyBE8PI6InSMAUl')
llama2 = LlamaAPI('LL-1JjExK2KMuL9qH30YOKdUvtk98gtZ0umxNmT8IOoWXweIqGiWmyBE8PI6InSMAUl')

In [ ]:
from llamaapi import LlamaAPI
import pandas as pd
import json

# Initialize the Llama API with your API token
# llama = LlamaAPI('your_llama_api_token')

def get_response_from_llama(llama_instance, prompt, conversation_history=None):
    # Convert conversation history to proper format (only string contents)


    # Build the API request JSON with the formatted conversation history and new prompt

    if conversation_history is not None:
      conversation_history=list(conversation_history)
      messages = [{"role": "system", "content": ""}]
      for entry in conversation_history:
        messages.append({"role": "user", "content": str(entry)})

    messages.append({"role": "user", "content": str(prompt)})

    api_request_json = {
        "model": "llama-13b-chat",
        "messages": [{"role": "user", "content": str(prompt)}]
    }

    # Execute the API request
    response = llama_instance.run(api_request_json)

    # Parse and print the JSON response
    if response.ok:
        return json.dumps(response.json(), indent=2)
    else:
        return "Error in Llama API response"



def main():
    # Load the prompts from the CSV file
    df = pd.read_csv('./generated_prompts_demo.csv')

    # Initialize a new column for the responses
    df['response'] = ''

    # Process each prompt and get responses from Llama
    for index, row in df.iterrows():


        Init_prompt_llama1 = f"The scenario is:{row['scenario']}, You are to act as a particular character. The following is the description of your character:{row['character1']}, Your goals are as follows:{row['goal1']}. You will be interacting with another character. You need to negotiate and come to a conclusion in exactly 5 iterations excluding this one. Do not start generating the conversation until you are given the keyword START. You will be starting the conversation and you will be allowed to generate 5 responses including the starting in an alternate back-and-forth fashion with your opponent. You must come to a conclusion in the conversation by your 5th response."
        Init_prompt_llama2 = f"The scenario is:{row['scenario']}, You are to act as a particular character. The following is the description of your character:{row['character2']}, Your goals are as follows:{row['goal2']}. You will be interacting with another character. You need to negotiate and come to a conclusion in exactly 5 iterations excluding this one. Your opponent will be starting the conversation. Do not start generating your response until you recieve the first conversation statement from your opponent. After this you will be allowed to generate a maximum of 5 responses and you must come to a conclusion in the conversation by your 5th response."
        init_response1 = get_response_from_llama(llama1,Init_prompt_llama1)
        init_response2 = get_response_from_llama(llama2,Init_prompt_llama2)

        conversation_history1=Init_prompt_llama1+"\n"+init_response1
        conversation_history2=Init_prompt_llama2+"\n"+init_response2

        llama1_response=get_response_from_llama(llama1,f"START",conversation_history1)
        response=llama1_response
        conversation_history1=conversation_history1+"\n"+llama1_response



        for i in range(4):
          llama2_response=get_response_from_llama(llama2,llama1_response,conversation_history2)
          response=response+"\n"+llama2_response
          conversation_history2=conversation_history2+"\n"+llama1_response+"\n"+llama2_response
          llama1_response=get_response_from_llama(llama1,llama2_response,conversation_history1)
          response=response+"\n"+llama1_response
          conversation_history1=conversation_history1+"\n"+llama2_response+"\n"+llama1_response


        llama2_response=get_response_from_llama(llama2,llama2_response,conversation_history2)
        response=response+"\n"+llama2_response
        conversation_history2=conversation_history2+"\n"+llama1_response+"\n"+llama2_response

        df.at[index, 'response'] = response

    # Save the updated DataFrame back to the CSV
    df.to_csv('./generated_response.csv', index=False)

if __name__ == "__main__":
    main()
